In [8]:
import requests
from bs4 import BeautifulSoup
import json
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

https://deliveroo.fr/fr/restaurants/paris/Aligre?geohash=u09tvuwxwxhq

In [9]:
#fonction qui recupere les tags avec comme attribut data-ar-restaurant-id
def restaurants_id (tag):
    return tag.has_attr('data-reactid')

In [48]:
#Fonction qui recupere les donnees des restaurants sur une page alloresto et les ajoute à une liste
def recuperer_liste_restaurants(link, adresse, liste_restaurants, liste_plats):
    
    # Create a new instance of the Firefox driver
    driver = webdriver.Firefox()

    # go to the page
    driver.get(link)

    #recuperer la liste des restaurants poour une adresse donnée
    print("1")
    inputSearch = driver.find_element_by_name("address_search")
    
    print("2")
    inputSearch.send_keys(adresse)
    
    print("3")
    inputSearch.send_keys(Keys.RETURN);
    print("4")
    
    # attendre que la page charge les restaurant supplémentaires
    WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CLASS_NAME, 'address-search--confirm')))
    print("5")
    
    #On va à la page des restaurants 
    inputElement = driver.find_element_by_css_selector("button.address-search--confirm")
    print("6")
    #print(inputElement)
    inputElement.click()
    # attendre que la page charge les restaurant supplémentaires
    WebDriverWait(driver,20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'restaurant-index-page-tile')))
    print("7")
    
    print(driver.page_source)
    
    
    #req = requests.get(link)
    #codeSource = req.text
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    #driver.quit()
    restaurants = soup.find_all('li', {"class" : "restaurant-index-page-tile"})
    #TODO: A retirer
    restaurants = restaurants[1:4]
    print (restaurants)
    
    for restaurant in restaurants:
        type(restaurant)
        restaurant = str(restaurant) 
        soup_r = BeautifulSoup(restaurant, 'html.parser')

        #link
        link_r = 'https://deliveroo.fr/fr' + soup_r.find("a")["href"]

        #On vérifie que le restaurant n'est pas déjà dans la liste
        isInList = False
        for r in liste_restaurants:
            if r["link"]==link:
                isInList = True

        if not isInList:

            #id
            id_restaurant = soup_r.find('li')['data-reactid']

            #name
            name = soup_r.find("h3").text.replace("\n","")
            
            #specialties
            specialties = soup_r.find("p").text.split(" ・ ")
            
            
            #On recupere les infos issues de la page du resto
            #driver.find_element_by_tag_name('body').send_keys(Keys.COMMAND + 't') 
            #driver.get(link_r)
            #codeSource = driver.page_source
            #driver.find_element_by_tag_name('body').send_keys(Keys.COMMAND + 'w') 
            codeSource = requests.get(link_r).text
            soup_page_restaurant = BeautifulSoup(codeSource, 'html.parser')
            #soup_page_restaurant = BeautifulSoup(codeSource, 'html.parser')
            
            #logo
            logo = ""
            try:
                logo = soup_page_restaurant.find("img", {"class": "restaurant__image"})['style']
            except:
                pass
            
            #address
            address = ""
            try:
                address = soup_page_restaurant.find("small", {"class": "address"}).text
            except:
                pass
            
            #phone
            phone = ""
            try:
                phone = soup_page_restaurant.find("small", {"class": "phone"}).text
            except:
                pass
            
            #description
            description = ""
            try:
                description = soup_page_restaurant.find("div", {"class": "restaurant__description"}).text
            except:
                pass
            
            #work_time
            work_time = ""
            try:
                work_time = soup_page_restaurant.find("small", {"class": "opening-hours"}).text
            except:
                pass

            #print(type(name))
            objet = {
                "id" : id_restaurant,
                "name" : name,
                "logo" : logo,
                "address" : address,
                "phone" : phone,
                "description" : description,
                "specialties" : specialties,
                "work_time" : work_time,
                #"minimum_order" : minimum_order,
                "link" : link_r
            }

            liste_restaurants.append(objet)
            #Appel de la fonction ajoutant la liste des plats
            recuperer_liste_plats(codeSource, id_restaurant, liste_plats)

    #print(i)

In [54]:
#recuperer les donnes sur les plats d'un restaurant donne
def recuperer_liste_plats(codeSource, id_restaurant, liste_plats):
    soup_carte = BeautifulSoup(codeSource, 'html.parser')
    plats = soup_carte.find_all("li", {"class" : "menu-index-page__item"})
    
    for plat in plats:
        soup_p = BeautifulSoup(str(plat) , 'html.parser')
        #print (soup_p.prettify())
        
        #id
        id_plat = soup_p.find("li")["data-reactid"]

        #On vérifie que le plat ajouté n'est pas deja dans la liste
        isInList = False
        for p in liste_plats:
            if p["id"]==id_plat:
                isInList = True

        if not isInList:
            #nom du plat
            name = soup_p.find("h6").text

            #logo
            logo = ""
            try:
                logo = soup_p.find("div", {"class" : "menu-index-page__item-image"})['src']
            except:
                pass
            
            #prix
            prix = soup_p.find("span" , {"class" : "menu-index-page__item-price"}).text
            prix = " ".join(prix.split())

            #description
            description = ""
            try:
                description = soup_p.find("p" , {"class" : "menu-index-page__item-desc"}).text
            except:
                pass                
                
            objet = {
                "id_restaurant" : id_restaurant,
                "id" : id_plat,
                "name" : name,
                "prix" : prix,
                "descrption" : description,
                "logo" : logo
            }
            #print(objet["prix"])

            liste_plats.append(objet)

In [55]:
liste_restaurants = []
liste_plats = []
link = "https://deliveroo.fr/fr/"
adresse="1 Avenue Daumesnil, 75012 Paris, France"
recuperer_liste_restaurants(link, adresse, liste_restaurants, liste_plats)

1
2
3
4
5
6
7
<html dir="ltr" lang="fr"><head>
  <meta charset="utf-8">
  <title>Livraison de repas Paris 12ème - Aligre en 30min - Commandez avec Deliveroo</title>
  <meta name="description" content="Faites-vous livrer les meilleurs restaurants de Paris 12ème - Aligre en moins de 30 minutes - 7 jours sur 7, de 8h à 23h.
">

  <meta name="viewport" content="width=device-width, initial-scale=1">
  <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
<script type="text/javascript" async="" src="https://www.google-analytics.com/plugins/ua/ecommerce.js"></script><script async="" src="https://www.google-analytics.com/analytics.js"></script><script src="https://connect.facebook.net/signals/config/1666308043654355?v=2.7.18" async=""></script><script async="" src="https://connect.facebook.net/en_US/fbevents.js"></script><script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"76617ea6ff","applicationI

In [56]:
liste_restaurants

[{'address': '1 rue des Francs Bourgeois, Paris, 75004',
  'description': 'Glace italienne d’Excellence, 100% issue de produits naturels et d’ingrédients sélectionnés, sans colorant, ni arôme artificiel. Découvrez nos coffrets isothermes et nos Macarons al Gelato.Read more',
  'id': '.1gid8uq3ri8.7.0.1.$13861',
  'link': 'https://deliveroo.fr/fr/menu/paris/4eme-hotel-de-ville/amorino-francs-bourgeois?day=today&rpos=1&time=ASAP',
  'logo': '',
  'name': 'Amorino Place des Vosges',
  'phone': '+33682318455',
  'specialties': ['Glaces', 'Dessert', 'Italien', '€€'],
  'work_time': "Ouvert jusqu'à 23:00"},
 {'address': '101 RUE DE CHARONNE, Paris, 75011',
  'description': 'La référence en terme de bièreRead more',
  'id': '.1gid8uq3ri8.7.0.1.$34581',
  'link': 'https://deliveroo.fr/fr/menu/paris/11eme-voltaire/paris-saint-biere?day=today&rpos=2&time=ASAP',
  'logo': '',
  'name': 'Paris Saint Bière',
  'phone': '+33143480711',
  'specialties': ['Français', 'Bière', 'Belge', '€'],
  'work_ti

In [57]:
len(liste_restaurants)

3

In [58]:
len(liste_plats)

269

In [59]:
liste_plats

[{'descrption': '1 à 2 parfums au choix',
  'id': '.r7kd3cypz4.2.4.0.2:$226988.2.$1721951',
  'id_restaurant': '.1gid8uq3ri8.7.0.1.$13861',
  'logo': '',
  'name': 'Coffret 550ml',
  'prix': '14,00 €'},
 {'descrption': 'de 1 à 3 parfums au choix',
  'id': '.r7kd3cypz4.2.4.0.2:$226988.2.$1721955',
  'id_restaurant': '.1gid8uq3ri8.7.0.1.$13861',
  'logo': '',
  'name': 'Coffret 1100ml',
  'prix': '25,00 €'},
 {'descrption': '',
  'id': '.r7kd3cypz4.2.4.0.2:$226988.2.$1406471',
  'id_restaurant': '.1gid8uq3ri8.7.0.1.$13861',
  'logo': '',
  'name': '4 cornets vides',
  'prix': '1,00 €'},
 {'descrption': '',
  'id': '.r7kd3cypz4.2.4.0.2:$226988.2.$1406472',
  'id_restaurant': '.1gid8uq3ri8.7.0.1.$13861',
  'logo': '',
  'name': '4 petits pots vides',
  'prix': '1,00 €'},
 {'descrption': 'de 1 à 3 parfums au choix',
  'id': '.r7kd3cypz4.2.4.0.2:$185890.2.$1406491',
  'id_restaurant': '.1gid8uq3ri8.7.0.1.$13861',
  'logo': '',
  'name': 'Coppa Grande',
  'prix': '6,90 €'},
 {'descrption': 'd